# 01 - Data and Data import

In this first section we will detail everything about the dataset and the pre-preparation we had to do to it in order to allow us to use it for our project. The [data](https://www.simpleweb.org/wiki/index.php/SSH_datasets) that we used was a selection of SSH logs from the linked web page.

In [3]:
import pandas as pd
import requests
import tarfile
import zipfile
from urllib.request import urlopen
from io import StringIO
import datetime as dt

In [12]:
global ip
text_file = urlopen("https://github.com/Galeforse/DST-Assessment-03/raw/master/Data/anon_ips_list2.txt")
ip = text_file.read().decode('utf-8').split(',')
print(len(ip))
ip[0]

73


'161.166.1.22'

In [9]:
def log_import(x):
    global ip
    y = str(x)
    root = "https://github.com/Galeforse/DST-Assessment-03/raw/master/Data/log_files/"
    df1 = pd.read_csv(root + y +".log.gz",names=["log"])   
    df1.insert(0, "anon_ip",ip[x-1], allow_duplicates = True)
    return(df1)

In [10]:
log_import(1)

,anon_ip,log
0,161.166.1.22,Jan 5 08:13:51 161.166.1.22 sshd[42590]: pam_...
1,161.166.1.22,Jan 5 08:13:53 161.166.1.22 sshd[42590]: Fail...
2,161.166.1.22,Jan 5 08:13:55 161.166.1.22 sshd[42590]: Fail...
3,161.166.1.22,Jan 5 08:13:57 161.166.1.22 sshd[42590]: Fail...
4,161.166.1.22,Jan 5 08:14:00 161.166.1.22 sshd[42590]: Fail...
...,...,...
113063,161.166.1.22,Feb 28 23:01:15 161.166.1.22 sshd[52192]: Rece...
113064,161.166.1.22,Feb 28 23:01:15 161.166.1.22 sshd[52192]: pam_...
113065,161.166.1.22,Feb 28 23:01:15 161.166.1.22 sshd[52192]: pam_...
113066,161.166.1.22,Feb 28 23:01:15 161.166.1.22 sshd[52218]: Acce...


In [12]:
start = dt.datetime.now()
df1 = log_import(1)
df2 = log_import(2)
df3 = log_import(3)
df4 = log_import(4)
df5 = log_import(5)
df6 = log_import(6)
df7 = log_import(7)
df8 = log_import(8)
df9 = log_import(9)
print("Imported data in: "+str(dt.datetime.now()-start))

In [13]:
start = dt.datetime.now()
df10 = log_import(10)
df11 = log_import(11)
df12 = log_import(12)
df13 = log_import(13)
df14 = log_import(14)
df15 = log_import(15)
df16 = log_import(16)
df17 = log_import(17)
df18 = log_import(18)
df19 = log_import(19)
print("Imported data in: "+str(dt.datetime.now()-start))

In [14]:
start = dt.datetime.now()
df20 = log_import(20)
df21 = log_import(21)
df22 = log_import(22)
df23 = log_import(23)
df24 = log_import(24)
df25 = log_import(25)
df26 = log_import(26)
df27 = log_import(27)
df28 = log_import(28)
df29 = log_import(29)
print("Imported data in : "+str(dt.datetime.now()-start))

In [15]:
start = dt.datetime.now()
df30 = log_import(30)
df31 = log_import(31)
df32 = log_import(32)
df33 = log_import(33)
df34 = log_import(34)
df35 = log_import(35)
df36 = log_import(36)
df37 = log_import(37)
df38 = log_import(38)
df39 = log_import(39)
print("Imported data in: "+str(dt.datetime.now()-start))

In [16]:
start = dt.datetime.now()
df40 = log_import(40)
df41 = log_import(41)
df42 = log_import(42)
df43 = log_import(43)
df44 = log_import(44)
df45 = log_import(45)
df46 = log_import(46)
df47 = log_import(47)
df48 = log_import(48)
df49 = log_import(49)
print("Imported data in: "+str(dt.datetime.now()-start))

In [17]:
start = dt.datetime.now()
df50 = log_import(50)
df51 = log_import(51)
df52 = log_import(52)
df53 = log_import(53)
df54 = log_import(54)
df55 = log_import(55)
df56 = log_import(56)
df57 = log_import(57)
df58 = log_import(58)
df59 = log_import(59)
print("Imported data in: "+str(dt.datetime.now()-start))

In [18]:
start = dt.datetime.now()
df60 = log_import(60)
df61 = log_import(61)
df62 = log_import(62)
df63 = log_import(63)
df64 = log_import(64)
df65 = log_import(65)
df66 = log_import(66)
df67 = log_import(67)
df68 = log_import(68)
df69 = log_import(69)
df70 = log_import(70)
df71 = log_import(71)
df72 = log_import(72)
df73 = log_import(73)
print("Imported data in: "+str(dt.datetime.now()-start))

In [20]:
dftest = pd.concat([df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df21,df22,df23,df24,df25,df26,df27,df28,df29,df30,
                       df31,df32,df33,df34,df35,df36,df37,df38,df39,df40,df41,df42,df43,df44,df45,df46,df47,df48,df49,df50,df51,df52,df53,df54,df55,df56,df57,df58,
                       df59,df60,df61,df62,df63,df64,df65,df66,df67,df68,df69,df70,df71,df72,df73])

In [21]:
dftest = dftest.reset_index()
dftest = dftest.drop(columns=["index"],axis=1)
dftest

,anon_ip,log
0,161.166.1.23,Jan 5 03:23:40 161.166.1.23 sshd[27076]: pam_...
1,161.166.1.23,Jan 5 03:23:42 161.166.1.23 sshd[27076]: Fail...
2,161.166.1.23,Jan 5 03:23:44 161.166.1.23 sshd[27076]: Fail...
3,161.166.1.23,Jan 5 03:23:47 161.166.1.23 sshd[27076]: Fail...
4,161.166.1.23,Jan 5 03:23:49 161.166.1.23 sshd[27076]: Fail...
...,...,...
15880517,161.166.236.154,Feb 23 05:20:38 161.166.236.154 sshd[16644]: I...
15880518,161.166.236.154,Feb 23 05:20:41 161.166.236.154 sshd[16662]: I...
15880519,161.166.236.154,Feb 23 05:20:44 161.166.236.154 sshd[16708]: I...
15880520,161.166.236.154,Feb 23 06:29:04 161.166.236.154 sshd[1282]: Us...


In [40]:
start = dt.datetime.now()
dftest.to_csv(path_or_buf="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/Data/master_log.csv.gz",index=False,compression="gzip")
print("Log written to czv.gz in: "+str(dt.datetime.now()-start))

In [41]:
smp_size = round(len(dftest)/10)
dftest_10 = dftest.sample(n=smp_size,random_state=7)
dftest_10 = dftest_10.sort_index()
dftest_10 = dftest_10.reset_index()
dftest_10 = dftest_10.drop(columns=["index"],axis=1)

In [42]:
start = dt.datetime.now()
dftest_10.to_csv(path_or_buf="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/Data/master_log_10.csv.gz",index=False,compression="gzip")
print("Log written to czv.gz in: "+str(dt.datetime.now()-start))

In [43]:
smp_size = round(len(dftest)/20)
dftest_5 = dftest.sample(n=smp_size,random_state=7)
dftest_5 = dftest_5.sort_index()
dftest_5 = dftest_5.reset_index()
dftest_5 = dftest_5.drop(columns=["index"],axis=1)

In [44]:
start = dt.datetime.now()
dftest_5.to_csv(path_or_buf="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/Data/master_log_5.csv.gz",index=False,compression="gzip")
print("Log written to czv.gz in: "+str(dt.datetime.now()-start))

In [22]:
smp_size = round(len(dftest)/2)
dftest_50 = dftest.sample(n=smp_size,random_state=7)
dftest_50 = dftest_50.sort_index()
dftest_50 = dftest_50.reset_index()
dftest_50 = dftest_50.drop(columns=["index"],axis=1)

In [26]:
start = dt.datetime.now()
dftest_50.to_csv(path_or_buf="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/Data/master_log_50.csv.gz",index=False,compression="gzip")
print("Log written to czv.gz in: "+str(dt.datetime.now()-start))

In [24]:
smp_size = round((len(dftest)*4)/10)
dftest_40 = dftest.sample(n=smp_size,random_state=7)
dftest_40 = dftest_40.sort_index()
dftest_40 = dftest_40.reset_index()
dftest_40 = dftest_40.drop(columns=["index"],axis=1)

In [27]:
start = dt.datetime.now()
dftest_40.to_csv(path_or_buf="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/Data/master_log_40.csv.gz",index=False,compression="gzip")
print("Log written to czv.gz in: "+str(dt.datetime.now()-start))